INSTRUCTIONS

1. open file in Jupyter Notebook
2. put excel sheet in script directory and store name of file in "excel_sheet" variable
3. install packages in terminal ('pip install requests os pandas plotly')
4. get API key from GCP Geocoder API (https://developers.google.com/maps/documentation/geocoding/start)
5. store API key as environment variable w/ the name "GM_API_KEY"
6. store list of desired items in "items" variable
7. run displayDeliveries(array)

In [ ]:
excel_name = 'Romania 2017-2022.xlsx'
items = ['ANELLO TEN INTERNO 45X62X8 NBR','KIT SEAL CRANE D16 21-10K55']

In [1]:
import pandas as pd
import os
import requests
import plotly.graph_objects as go

In [2]:
df = pd.read_excel(excel_name)

In [3]:
# GOOGLE CLOUD PLATFORM GEOCODER API KEY (stored as environment variable)
API_KEY = os.environ.get('GM_API_KEY')
base_URL = 'https://maps.googleapis.com/maps/api/geocode/json'

def getLL(address):
    params = {
        'key': API_KEY,
        'address': address.replace(' ','+')
    }

    response = requests.get(base_URL, params=params)
    data = response.json()
    if data['status'] == 'OK':
        result = data['results'][0]
        location = result['geometry']['location']
        city = list(filter(lambda comp: 'locality' in comp['types'], result['address_components']))
        city = city[0]['long_name'] if len(city) else None
        location['city'] = city

    return location

In [4]:
def getDeliveries(items):
    queryStr = ' or '.join(["ItemName == '{item}'".format(item=item) for item in items])
    deliveries = df.query(queryStr)[['ItemName','Del.Addr','OrdQty']].reset_index()

    for row in range(deliveries.shape[0]):
        delivery = deliveries.iloc()[row]

        coords = getLL(delivery['Del.Addr'])

        deliveries.at[row,'lat'] = coords['lat']
        deliveries.at[row,'lng'] = coords['lng']

        deliveries.at[row,'text'] = 'Item: {item}<br>City: {city}<br>Volume: {qty}'.format(
                            item=delivery['ItemName'],
                            city=coords['city'], 
                            qty=delivery['OrdQty'])
    return deliveries

def getGeoObj(df,item,index):
    minQty = min(df['OrdQty'].tolist())
    maxQty = max(df['OrdQty'].tolist())

    deliveries = df.query("ItemName == '{item}'".format(item=item))
        
    geoObj = go.Scattergeo(
        lon = deliveries['lng'],
        lat = deliveries['lat'],
        text = deliveries['text'],
        mode = 'markers',
        marker = dict(
            color = deliveries['OrdQty'],
            colorscale = ['red','blue'],
            cauto = False,
            autocolorscale = False,
            cmin = minQty,
            cmax = maxQty,
            colorbar = None if index > 0 else dict(
                title = 'Volume',
                titleside = 'top',
                tickmode = 'array',
            )
        )
    )

    geoObj['marker_symbol'] = symbols[index%len(symbols)]
    geoObj['name'] = item

    return geoObj

def displayDeliveries(items):
    deliveries = getDeliveries(items)

    fig = go.Figure(data=[getGeoObj(deliveries,item,index) for index,item in enumerate(items)])

    fig.update_geos(
        lataxis_range=[43, 49],
        lonaxis_range=[20, 30],
    )

    title = 'Delivery Locations for '
    title += '{amount} items'.format(amount=len(items)) if len(items) > 1 else items[0]

    fig.update_layout(
        title=title,
        geo_scope='europe',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=-1
        )
    )

    fig.show()

In [5]:
# items = list(set(df['ItemName'].head(100)))
symbols = ['circle','square','diamond','cross','x','triangle-up',
    'triangle-down','triangle-left','triangle-right']

In [8]:
displayDeliveries(items)